In [1]:
from torch_snippets import *
import selectivesearch
from skimage.segmentation import felzenszwalb

/home/diego/anaconda3/envs/PyTorch_GPU/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/diego/anaconda3/envs/PyTorch_GPU/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
# !wget https://www.dropbox.com/s/l98leemr7r5stnm/Hemanvi.jpeg
img = read('Hemanvi.jpeg')

In [3]:
# segment_fz = felzenszwalb(img, scale=200)

In [4]:
# subplots([img, segment_fz], titles = ['Original Image', 'Image post \n felzenswalb segmentation'], sz=10, nc=2)

In [5]:
def extract_candidates(img):
    img_lbl, regions = selectivesearch.selective_search(img, scale=200, min_size=100)
    img_area = np.prod(img.shape[:2])
    candidates = []
    for r in regions:
        if r['rect'] in candidates: continue
        if r['size'] < (0.05*img_area): continue
        if r['size'] > (1*img_area): continue
        x, y, w, h = r['rect']
        candidates.append(list(r['rect']))
    return candidates

In [6]:
# img = read('Hemanvi.jpeg', 1)
# candidates = extract_candidates(img)
# show(img, bbs=candidates)

In [7]:
def get_iou(boxA, boxB, epsilon=1e-5):
    x1 = max(boxA[0], boxB[0])
    y1 = max(boxA[1], boxB[1])
    x2 = min(boxA[2], boxB[2])
    y2 = min(boxA[3], boxB[3])
    width = (x2 - x1)
    height = (y2 -y1)
    if width<0 or height<0:
        return 0.0
    area_overlap = width*height
    area_a = (boxA[2]-boxA[0])*(boxA[3]-boxA[1])
    area_b = (boxB[2]-boxB[0])*(boxB[3]-boxB[1])
    area_combined =  area_a + area_b - area_overlap
    iou = area_overlap / (area_combined + epsilon)
    return iou